In [5]:
%pip install gradio

  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
   ---------------------------------------- 0.0/46.9 MB ? eta -:--:--
   -- ------------------------------------- 2.9/46.9 MB 16.8 MB/s eta 0:00:03
   ----- ---------------------------------- 6.8/46.9 MB 16.8 MB/s eta 0:00:03
   -------- ------------------------------- 10.2/46.9 MB 17.2 MB/s eta 0:00:03
   ------------ --------------------------- 14.9/46.9 MB 18.1 MB/s eta 0:00:02
   ---------------- ----------------------- 19.4/46.9 MB 18.6 MB/s eta 0:00:02
   -------------------- ------------------- 23.6/46.9 MB 18.9 MB/s eta 0:00:02
   ----------------------- ---------------- 27.8/46.9 MB 18.9 MB/s eta 0:00:02
   --------------------------- ------------ 31.7/46.9 MB 19.0 MB/s eta 0:00:01
   ------------------------------ --------- 36.2/46.9 MB 19.2 MB/s eta 0:00:01
   ---------------------------------- ----- 40.4/46.9 MB 19.2 MB/s eta 0:00:01
   ------------------------------------- -- 43.5/46.9 MB 18.8 MB

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.28.1 which is incompatible.


In [ ]:
# ✅ Imports
import gradio as gr
from sentence_transformers import SentenceTransformer, util
from datetime import datetime
import requests
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# ✅ Load environment variables
load_dotenv()
API_KEY = os.getenv("OPENWEATHER_API_KEY")

# ✅ Load Crop Dataset and Train Model
df = pd.read_csv("Crop_recommendation.csv")

# Features and labels
X = df[["temperature", "humidity"]]
y = df["label"]

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2)

# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# ✅ NER Setup
ner_model_name = "Davlan/bert-base-multilingual-cased-ner-hrl"
tokenizer = AutoTokenizer.from_pretrained(ner_model_name)
ner_model = AutoModelForTokenClassification.from_pretrained(ner_model_name)
ner_pipeline = pipeline("ner", model=ner_model, tokenizer=tokenizer, grouped_entities=True)

# ✅ WSD Model and Sense Inventory
wsd_model = SentenceTransformer('all-MiniLM-L6-v2')
sense_inventory = {
    "plant_1": "a living organism like a crop or tree",
    "plant_2": "an industrial facility such as a pesticide plant",
    "spray_1": "the act of spraying liquid on crops",
    "spray_2": "a physical pesticide product in a bottle"
}

def disambiguate_word(context_sentence, word, sense_inventory):
    senses = [k for k in sense_inventory if k.startswith(word)]
    if not senses:
        return None

    context_embedding = wsd_model.encode(context_sentence, convert_to_tensor=True)
    best_sense, best_score = None, -1
    for sense_key in senses:
        gloss = sense_inventory[sense_key]
        gloss_embedding = wsd_model.encode(gloss, convert_to_tensor=True)
        score = util.pytorch_cos_sim(context_embedding, gloss_embedding).item()
        if score > best_score:
            best_score = score
            best_sense = sense_key
    return best_sense

# ✅ Extract location from query using NER
def extract_location(text):
    entities = ner_pipeline(text)
    for ent in entities:
        if ent["entity_group"] == "LOC":
            return ent["word"]
    return None

# ✅ Get Weather Info
# def get_weather_data(location, api_key):
#     try:
#         url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
#         response = requests.get(url)
#         data = response.json()
#         if data.get("cod") != 200:
#             return None, None
#         weather_desc = data['weather'][0]['description']
#         temperature = data['main']['temp']
#         humidity = data['main']['humidity']
#         return weather_desc, temperature , humidity
#     except Exception as e:
#         print(f"Error fetching weather: {e}")
#         return None, None

# # ✅ Predict crop using only city
# def predict_crop_from_city(city):
#     weather_desc, temp , humidity = get_weather_data(city, API_KEY)
    
#     if temp is None:
#         return f"❌ Couldn't fetch weather info for '{city}'. Please try another location."


#     # Default values from dataset or assumption
#     # avg_N = 90
#     # avg_P = 42
#     # avg_K = 43
#     # avg_ph = 6.5
#     # avg_rainfall = 100

#     #input_data = [[avg_N, avg_P, avg_K, temp, humidity, avg_ph, avg_rainfall]]
#     input_data = [[ temp, humidity]]
#     crop_encoded = model.predict(input_data)[0]
#     crop_label = label_encoder.inverse_transform([crop_encoded])[0]

#     return f"🌤️ Weather in {city}: {weather_desc}, {temp}°C\n🌱 Recommended crop: {crop_label}"


# def recommend_crop(city):
#     weather_desc, temp , humidity = get_weather_data(city, API_KEY)

    
#     # Predict crop
#     input_data = np.array([[temp, humidity]])
#     crop_encoded = model.predict(input_data)[0]
#     crop_label = label_encoder.inverse_transform([crop_encoded])[0]

#     return f"  Weather Description: {weather_desc}\n 🌡 Temperature: {temp}°C\n💧 Humidity: {humidity}%\n🌱 Recommended crop: {crop_label} "

# # --------------------------------
# # ✅ Gradio UI
# demo = gr.Interface(
#     fn=recommend_crop,
#     inputs=gr.Textbox(label="Enter your city"),
#     outputs="text",
#     title="🌾 Crop Recommendation Bot",
#     description="Enter your city name to get real-time crop suggestions based on weather data."
# )

# demo.launch()



Device set to use cpu
c:\Users\kurup\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\pipelines\token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
